<a href="https://colab.research.google.com/github/fvndnv/vdieo-analysis/blob/master/another_video_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/ucfTrainTestlist/trainlist01.txt')

In [0]:
df.head()

,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c01.avi 1


In [0]:
import math
import os
import pandas as pd
import numpy as np

In [0]:
import keras
from keras.layers import Dense,Dropout,InputLayer
from skimage.transform import resize
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

In [0]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from keras.utils import np_utils

In [0]:
f=open("/content/drive/My Drive/ucfTrainTestlist/trainlist01.txt", "r")
temp=f.read()
videos=temp.split('\n')
train=pd.DataFrame()
train['videogames']=videos
train=train[:-1]
train.head()

,videogames
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1


In [0]:
## This is how the names of videos are given in the .txt file. It is not properly aligned and we will need to preprocess it. 
## Before that, let’s create a similar dataframe for test videos as well:

In [0]:
f=open("/content/drive/My Drive/ucfTrainTestlist/testlist01.txt", "r")
temp=f.read()
videos=temp.split('\n')
test=pd.DataFrame()
test['videogames']=videos
test=test[:-1]
test.head()

,videogames
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


In [0]:
### Next, we will add the tag of each video (for both training and test sets).
### Did you notice that the entire part before the ‘/’ in the video name represents the tag of the video? Hence, we will split the entire string on ‘/’ and select the tag for all the videos:

In [0]:
train.shape,  train.shape[0]

((9537, 1), 9537)

In [0]:
# creating tags for training videos
tttrain_video_tag=[]
for i in range(train.shape[0]):
  tttrain_video_tag.append(train['videogames'][i].split('/'))

tttrain_video_tag[:2]

[['ApplyEyeMakeup', 'v_ApplyEyeMakeup_g08_c01.avi 1'],
 ['ApplyEyeMakeup', 'v_ApplyEyeMakeup_g08_c02.avi 1']]

In [0]:
# creating tags for training videos
train_video_flag=[]
for i in range(train.shape[0]):
  train_video_flag.append(train['videogames'][i].split('/')[0])
train['tag']=train_video_flag

### creating tags for test videos

test_video_flag=[]
for i in range(test.shape[0]):
  test_video_flag.append(test['videogames'][i].split('/')[0])
test['tag']=test_video_flag
 

In [0]:
train.shape,    test.shape

((9537, 2), (3783, 2))

In [0]:
train.columns, test.columns

(Index(['videogames', 'tag'], dtype='object'),
 Index(['videogames', 'tag'], dtype='object'))

In [0]:
# So what’s next? Now, we will extract the frames from the training videos which will be used to train the model. I will be storing all the frames in a folder named train_1.

# So, first of all, make a new folder and rename it to ‘train_1’ and then follow the code given below to extract frames:

In [0]:
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['videogames'][i]
    cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename ='train_1/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()


100%|██████████| 9537/9537 [00:00<00:00, 19470.68it/s]

This will take some time as there are more than 9,500 videos in the training set. Once the frames are extracted, we will save the name of these frames with their corresponding tag in a .csv file. Creating this file will help us to read the frames which we will see in the next section.

In [0]:
images = glob("train_1/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i].split('/')[1])
    # creating the class of image
    train_class.append(images[i].split('/')[1].split('_')[1])
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('/content/drive/My Drive/ucfTrainTestlist/train_new.csv',header=True, index=False)


0it [00:00, ?it/s]


In [0]:
t=pd.read_csv("/content/drive/My Drive/ucfTrainTestlist/train_new.csv")
t.head()

,image,class


So far, we have extracted frames from all the training videos and saved them in a .csv file along with their corresponding tags. It’s now time to train our model which we will use to predict the tags for videos in the test set.

Training the Video Classification Model
It’s finally time to train our video classification model! I’m sure this is the most anticipated section of the tutorial. I have divided this step into sub-steps for ease of understanding:

Read all the frames that we extracted earlier for the training images
Create a validation set which will help us examine how well our model will perform on unseen data
Define the architecture of our model
Finally, train the model and save its weights

Reading all the video frames
So, let’s get started with the first step where we will extract the frames. We will import the libraries first:

In [1]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
train = pd.read_csv('UCF/train_new.csv')
train.head()

This is how the first five rows look like. We have the corresponding class or tag for each frame. Now, using this .csv file, we will read the frames that we extracted earlier and then store those frames as a NumPy array:

In [0]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

Output: (73844, 224, 224, 3)

We have 73,844 images each of size (224, 224, 3). Next, we will create the validation set.

In [0]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)


Here, stratify = y (which is the class or tags of each frame) keeps the similar distribution of classes in both the training as well as the validation set.

Remember – there are 101 categories in which a video can be classified. So, we will have to create 101 different columns in the target, one for each category. We will use the get_dummies() function for that:

In [0]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

Defining the architecture of the video classification model
Since we do not have a very large dataset, creating a model from scratch might not work well. So, we will use a pre-trained model and take its learnings to solve our problem.

For this particular dataset, we will be using the VGG-16 pre-trained model. Let’s create a base model of the pre-trained model:

In [0]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

This model was trained on a dataset that has 1,000 classes. We will fine tune this model as per our requirement. include_top = False will remove the last layer of this model so that we can tune it as per our need.

Now, we will extract features from this pre-trained model for our training and validation images:

In [0]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

Output: (59075, 7, 7, 512)

We have 59,075 images in the training set and the shape has been changed to (7, 7, 512) since we have passed these images through the VGG16 architecture. Similarly, we will extract features for validation frames:

In [0]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

Output: (14769, 7, 7, 512)

There are 14,769 images in the validation set and the shape of these images has also changed to (7, 7, 512). We will use a fully connected network now to fine-tune the model. This fully connected network takes input in single dimension. So, we will reshape the images into a single dimension:

In [0]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(59075, 7*7*512)
X_test = X_test.reshape(14769, 7*7*512)

It is always advisable to normalize the pixel values, i.e., keep the pixel values between 0 and 1. This helps the model to converge faster.

In [0]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

Next, we will create the architecture of the model. We have to define the input shape for that. So, let’s check the shape of our images:

In [0]:
# shape of images
X_train.shape

Output: (59075, 25088)

The input shape will be 25,088. Let’s now create the architecture:

In [0]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

Training the video classification model
We will now train our model using the training frames and validate the model using validation frames. We will save the weights of the model so that we will not have to retrain the model again and again.

So, let’s define a function to save the weights of the model:

In [0]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

We will decide the optimum model based on the validation loss. Note that the weights will be saved as weights.hdf5. You can rename the file if you wish. Before training the model, we have to compile it:

In [0]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [0]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)


I have trained the model for 200 epochs. To download the weights which I got after training the model, you can use this link.

We now have the weights which we will use to make predictions for the new videos. So, in the next section, we will see how well this model will perform on the task of video classification!

Evaluating our Video Classification Model
Let’s open a new Jupyter notebook to evaluate the model. The evaluation part can also be split into multiple steps to understand the process more clearly:

Define the model architecture and load the weights
Create the test data
Make predictions for the test videos
Finally, evaluate the model
 

Defining model architecture and loading weights
You’ll be familiar with the first step – importing the required libraries:

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

Next, we will define the model architecture which will be similar to what we had while training the model:



In [0]:
base_model = VGG16(weights='imagenet', include_top=False)


This is the pre-trained model and we will fine-tune it next:



In [0]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))


Now, as we have defined the architecture, we will now load the trained weights which we stored as weights.hdf5:



In [0]:
model.load_weights("weights.hdf5")
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])


Make sure that the loss function, optimizer, and the metrics are the same as we used while training the model.



In [0]:
Creating the test data
You should have downloaded the train/test split files as per the official documentation of the UCF101 dataset. If not, download it from here. In the downloaded folder, there is a file named “testlist01.txt” which contains the list of test videos. We will make use of that to create the test data:

In [0]:
# getting the test list
f = open("testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

We now have the list of all the videos stored in a dataframe. To map the predicted categories with the actual categories, we will use the train_new.csv file:



In [0]:
# creating the tags
train = pd.read_csv('UCF/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

Now, we will make predictions for the videos in the test set.

Generating predictions for test videos
Let me summarize what we will be doing in this step before looking at the code. The below steps will help you understand the prediction part:

First, we will create two empty lists – one to store the predictions and the other to store the actual tags
Then, we will take each video from the test set, extract frames for this video and store it in a folder (create a folder named temp in the current directory to store the frames). We will remove all the other files from this folder at each iteration
Next, we will read all the frames from the temp folder, extract features for these frames using the pre-trained model, predict tags, and then take the mode to assign a tag for that particular video and append it in the list
We will append actual tags for each video in the second list
Let’s code these steps and generate predictions:

In [0]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])

This step will take some time as there are around 3,800 videos in the test set. Once we have the predictions, we will calculate the performance of the model.

 

Evaluating the model
Time to evaluate our model and see what all the fuss was about.

We have the actual tags as well as the tags predicted by our model. We will make use of these to get the accuracy score. On the official documentation page of UCF101, the current accuracy is 43.90%. Can our model beat that? Let’s check!

In [0]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100